In [1]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split() #1
    except AttributeError: # 不能调用.replace或.split方法
        pass # 假定field_names本就是标识符组成的序列
    field_names = tuple(field_names) #2
    
    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
            
    def __iter__(self): #4
        for name in self.__slots__:
            yield getattr(self, name)
            
    def __repr__(self): #5
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(
        __slots__ = field_names,
        __init__ = __init__,
        __iter__ = __iter__,
        __repr__ = __repr
    ) #6
    
    return type(cls_name, (object,), cls_attrs) #7

In [2]:
MyClass = type('MyClass', (MySuperClass, MyMixin), {'x': 42, 'x2': lambda self: self.x * 2})

NameError: name 'MySuperClass' is not defined

In [1]:
def entity(cls): #1
    for key, attr in cls.__dict__.items(): #2
        if isinstance(attr, Validated): #4
            type_name = type(attr).__name__
            attr.strorage_name = '_{}#{}'.format(type_name, key) #4
    return cls #5

In [1]:
import model_v6 as model
from model_v6 import entity

@entity
class LineItem:
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [3]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [4]:
dir(raisins)[:3]

['_NonBlank#1', '_Quantity#2', '_Quantity#3']

In [5]:
import evaltime

ModuleNotFoundError: No module named 'evaltime'

evalsupport.py：evaltime.py 导入的模块

In [6]:
print('<[100]> evalsupport module start')

def deco_alpha(cls):
    print('<[200]> deco_alpha')
    def inner_1(self):
        print('<[300]> deco_alpha:inner_1')
        
    cls.method_y = inner_1
    return cls

class MetaAleph(type):
    print('<[400]> MetaAleph body')
    
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')
        
        def inner_2(self):
            print('<[600]> MetaAleph.__init__:init_2')
            
        cls.method_z = inner_2
        
print('<[700]> evalsupport module end')

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end


evaltime.py：按顺序写出输出的序号标记 <[N]>

In [ ]:
from evalsupport import deco_alpha

print('<[1]> evaltime module start')


class ClassOne():
    print('<[2]> ClassOne body')
    
    def __init__(self):
        print('<[3]> ClassOne.__init__')
        
    def __del__(self):
        print('<[4]> ClassOne.__del_-')
        
    def method_x(self):
        print('<[5]> ClassOne.method_x')
        
    class ClassTwo(object):
        print('<[6]> ClassTwo body')
        

@deco_alpha
class ClassThree():
    print('<[7]> ClassThree body')
    
    def method_y(self):
        print('<[8]> ClassThree.method_y')
        
class ClassFour(ClassThree):
    print('<[9]> ClassFour body')
    
    def method_y(self):
        print('<[10]> ClassFour.method_y')
        
        
if __name__ == '__main__':
    print('<[11]> ClassOne tests', 30 * '.')
    one = ClassOne()
    one.method_x()
    print('<[12]> ClassThree tests', 30 * '.')
    three = ClassThree()
    three.method_y()
    print('<[13]> CLassFour tests', 30 * '.')
    four = ClassFour()
    four.method_y()
    
print('<[14]> evaltime module end')

In [7]:
import evaltime 

<[1]> evaltime module start
<[2]> ClassOne body
<[6]> ClassTwo body
<[7]> ClassThree body
<[200]> deco_alpha
<[9]> ClassFour body
<[14]> evaltime module end


In [8]:
'spam'.__class__

str

In [9]:
str.__class__

type

In [10]:
type.__class__

type

In [11]:
import collections

In [12]:
collections.Iterable.__class__

abc.ABCMeta

In [13]:
import abc

In [14]:
abc.ABCMeta.__class__

type

In [15]:
abc.ABCMeta.__mro__

(abc.ABCMeta, type, object)

In [ ]:
from evalsupport import deco_alpha
from evalsupport import MetaAleph

print('<[1]> evaltime_meta module start')

@deco_alpha
class ClassThree():
    print('<[2]> ClassThree body')
    
    def method_y(self):
        print('<[3]> ClassThree.method_y')
        
class ClassFour(ClassThree):
    print('<[4]> ClassFour body')
    
    def method_y(self):
        print('<[5]> ClassFour.method_y')

class ClassFive(metaclass=MetaAleph):
    print('<[6]> CLassFive body')
    
    def __init__(self):
        print('<[7]> ClassFive.__init__')
        
    def method_z(self):
        print('<[8]> ClassFive.method_z')
        
class ClassSix(ClassFive):
    print('<[9]> ClassSix body')
    def method_z(self):
        print('<[10]> ClassSix.method_z')
        
        
if __name__ == '__main__':
    print('<[11]> ClassThree tests', 30 * '.')
    three = ClassThree()
    three.method_y()
    print('<[12]> ClassFour tests', 30 * '.')
    four = ClassFour()
    four.method_y()
    print('<[13]> ClassFive tests', 30 * '.')
    five = ClassFive()
    five.method_z()
    print('<[14]> ClassSix tests', 30 * '.')
    six = ClassSix()
    six.method_z()
    
    
print('<[15]> evaltime_meta module end')

In [16]:
class EntityMeta(type):
    """元类，用于创建带有验证字段的业务实体"""
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict) #1
        for key, attr in attr_dict.items(): #2
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                
class Entity(metaclass=EntityMeta): #3
    """带有验证字段的业务实体"""

NameError: name 'Validated' is not defined

In [ ]:
import collections
class EntityMeta(type):
    """元类，用于创建带有验证字段的业务实体"""
    
    @classmethod
    def __prepare__(cls, name, bases):
        return collections.OrderedDict()
    
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        cls._field_names = []
        for key, attr in attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                cls._field_names.append(key)
                
class Entity(metaclass=EntityMeta): #3
    """带有验证字段的业务实体"""
    
    @classmethod
    def field_names(cls):
        for name in cls._field_names:
            yield name

In [19]:
help(__mro__)

NameError: name '__mro__' is not defined

In [20]:
str.__name__

'str'

In [21]:
LineItem.__name__

'LineItem'

In [23]:
LineItem().__name__

TypeError: __init__() missing 3 required positional arguments: 'description', 'weight', and 'price'

In [31]:
class A:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def f():
        print('f')

In [32]:
A().__name__

TypeError: __init__() missing 2 required positional arguments: 'a' and 'b'

In [34]:
x = A(1, 2)

In [35]:
x

In [36]:
x.__class__

__main__.A

In [37]:
x.__name__

AttributeError: 'A' object has no attribute '__name__'

In [38]:
x.__mro__

AttributeError: 'A' object has no attribute '__mro__'

In [39]:
def aa():
    print('aa')

In [40]:
aa.__name__

'aa'

In [41]:
aa.__class__

function

In [42]:
aa.__mro__

AttributeError: 'function' object has no attribute '__mro__'

In [43]:
str.__mro__

(str, object)

In [54]:
class F:
    class FF:
        class AA:
            def me(self):
                print(type(self).__qualname__)

In [56]:
x = F.FF.AA()

In [57]:
x.me()

F.FF.AA


In [47]:
x.__qualname__

AttributeError: 'F' object has no attribute '__qualname__'

In [48]:
x.FF

__main__.F.FF

In [49]:
a = F.FF()

In [50]:
a.__name__

AttributeError: 'FF' object has no attribute '__name__'

In [51]:
a.__mro__

AttributeError: 'FF' object has no attribute '__mro__'

In [52]:
a.qualname__

AttributeError: 'FF' object has no attribute 'qualname__'

In [53]:
a